In [1]:
# Lebanese restaurent in Los Angeles

# Import required libraries
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
import requests
from bs4 import BeautifulSoup
import io
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
from geopy.geocoders import Nominatim

In [2]:
# List of districts and neighborhoods of Los Angeles
# Get data from wikipedia page, process the data to extract table
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_districts_and_neighborhoods_of_Los_Angeles').text
soup = BeautifulSoup(website_url,'html')
# soup.prettify()

In [3]:
# print(soup.prettify())
neighbourhoods_list = soup.findAll('ul')  # in this result set, the useful lists are the 3rd and 4th
# Extracting neighborhoods' list
neighborhoods = []

for item in neighbourhoods_list[2].findAll('li'):
    neighborhoods.append(item.a.renderContents().decode("utf-8"))
    
for item in neighbourhoods_list[3].findAll('li'):
    neighborhoods.append(item.a.renderContents().decode("utf-8"))

print('List of Los Angeles neighborhoods:\n' + "\n".join(neighborhoods))

List of Los Angeles neighborhoods:
Angelino Heights
Arleta
Arlington Heights
Arts District
Atwater Village
Baldwin Hills
Baldwin Hills/Crenshaw
Baldwin Village
Baldwin Vista
Beachwood Canyon
Bel Air, Bel-Air or Bel Air Estates
Benedict Canyon
Beverly Crest
Beverly Glen
Beverly Grove
Beverly Hills Post Office
Beverly Park
Beverlywood
Boyle Heights
Brentwood
Brentwood Circle
Brentwood Glen
Broadway-Manchester
Brookside
Bunker Hill
Cahuenga Pass
Canoga Park
Canterbury Knolls
Carthay
Castle Heights
Central-Alameda
Central City
Century City
Chatsworth
Chesterfield Square
Cheviot Hills
Chinatown
Civic Center
Crenshaw
Crestwood Hills
Cypress Park
Del Rey
Downtown
Eagle Rock
East Gate Bel Air
East Hollywood
Echo Park
Edendale
El Sereno
Elysian Heights
Elysian Park
Elysian Valley
Encino
Exposition Park
Faircrest Heights
Fairfax
Fashion District
Filipinotown, Historic
Financial District
Florence
Flower District
Franklin Hills
Gallery Row
Garvanza
Glassell Park
Gramercy Park
Granada Hills
Green M

In [4]:
# Correcting specific neighborhoods names
for n, i in enumerate(neighborhoods):
    if i == '[33]':
        neighborhoods[n] = 'Pico Robertson'
    if i == 'Bel Air, Bel-Air or Bel Air Estates':
        neighborhoods[n] = 'Bel Air'

print('List of Los Angeles neighborhoods:\n' + "\n".join(neighborhoods))

# Forming the neighborhoods dataframe
columns = ['Neighborhood']
neighborhoods_df = pd.DataFrame(columns = columns)
neighborhoods_df['Neighborhood'] = neighborhoods
neighborhoods_df.head()

List of Los Angeles neighborhoods:
Angelino Heights
Arleta
Arlington Heights
Arts District
Atwater Village
Baldwin Hills
Baldwin Hills/Crenshaw
Baldwin Village
Baldwin Vista
Beachwood Canyon
Bel Air
Benedict Canyon
Beverly Crest
Beverly Glen
Beverly Grove
Beverly Hills Post Office
Beverly Park
Beverlywood
Boyle Heights
Brentwood
Brentwood Circle
Brentwood Glen
Broadway-Manchester
Brookside
Bunker Hill
Cahuenga Pass
Canoga Park
Canterbury Knolls
Carthay
Castle Heights
Central-Alameda
Central City
Century City
Chatsworth
Chesterfield Square
Cheviot Hills
Chinatown
Civic Center
Crenshaw
Crestwood Hills
Cypress Park
Del Rey
Downtown
Eagle Rock
East Gate Bel Air
East Hollywood
Echo Park
Edendale
El Sereno
Elysian Heights
Elysian Park
Elysian Valley
Encino
Exposition Park
Faircrest Heights
Fairfax
Fashion District
Filipinotown, Historic
Financial District
Florence
Flower District
Franklin Hills
Gallery Row
Garvanza
Glassell Park
Gramercy Park
Granada Hills
Green Meadows
Griffith Park
Hancock

,Neighborhood
0,Angelino Heights
1,Arleta
2,Arlington Heights
3,Arts District
4,Atwater Village


In [8]:
# Getting the geograpical coordinates of each neighborhood
latitudes = []
longitudes = []

for neigh in neighborhoods_df['Neighborhood']:
    location = None
    for k in range(10):
        try:
            address = '{}, Los Angeles, US'.format(neigh)
            geolocator = Nominatim(user_agent="LA_explorer")
            location = geolocator.geocode(address)
            latitude = location.latitude
            longitude = location.longitude
            latitudes.append(latitude)
            longitudes.append(longitude)
            print('The location coordinates of {} are {} and {}'.format(neigh, latitude, longitude))
            break
        except:
            pass
    if location is None:
        latitudes.append(0)
        longitudes.append(0)
print('Done!')

The location coordinates of Angelino Heights are 34.0702889 and -118.2547965
The location coordinates of Arleta are 34.2413266 and -118.4322047
The location coordinates of Arlington Heights are 34.0434937 and -118.3213735
The location coordinates of Arts District are 34.0412389 and -118.2344503
The location coordinates of Atwater Village are 34.1186975 and -118.2623924
The location coordinates of Baldwin Hills/Crenshaw are 34.0075684 and -118.3505956
The location coordinates of Beachwood Canyon are 34.1222919 and -118.3213845
The location coordinates of Bel Air are 34.0988833 and -118.4598811
The location coordinates of Benedict Canyon are 34.0447702 and -118.4000223
The location coordinates of Beverly Crest are 34.1167701 and -118.4322607
The location coordinates of Beverly Glen are 34.1077855 and -118.4456361
The location coordinates of Beverly Grove are 34.0760338 and -118.3699723
The location coordinates of Beverly Park are 34.06376935 and -118.26468959898818
The location coordinat

In [9]:
len(latitudes)

195

In [13]:
# Add the obtained coordinates to our dataframe
neighborhoods_df['Latitude'] = latitudes
neighborhoods_df['Longitude'] = longitudes
# Drop those neighborhoods for which we could not find the coordinates
neighborhoods_df = neighborhoods_df[[a and b for a, b in zip(neighborhoods_df['Latitude'] != 0, neighborhoods_df['Longitude'] != 0)]].reset_index(drop=True)
neighborhoods_df.head()

,Neighborhood,Latitude,Longitude
0,Angelino Heights,34.070289,-118.254796
1,Arleta,34.241327,-118.432205
2,Arlington Heights,34.043494,-118.321374
3,Arts District,34.041239,-118.234450
4,Atwater Village,34.118698,-118.262392


In [20]:
# get the address of Los Angeles
address = 'Los Angeles, US'

geolocator = Nominatim(user_agent="LA_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Los Angeles are {}, {}.'.format(latitude, longitude))

GeocoderQuotaExceeded: HTTP Error 429: Too Many Requests

In [21]:
# create map of Los Angeles using latitude and longitude values
map_LA = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, neighborhood in zip(neighborhoods_df['Latitude'], neighborhoods_df['Longitude'], neighborhoods_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_LA)  
    
map_LA